# 9. Training Two Separate DeBERTa Verifiers

In this notebook we train two specialized verifiers for our structured reasoning pipeline:

1. **Question Parsing (QP) Verifier**  
   - Input: raw question → serialized `question_parsing`  
   - Negatives: drop or shuffle constraints  
   - Model: `microsoft/deberta-v3-base`, 5 epochs, class-balanced  

2. **Chain-of-Thought (CoT) Verifier**  
   - Input: question + conditions + CoT → serialized `cot_parsing`  
   - Negatives: flip verification, swap/drop evidence  
   - Model: `microsoft/deberta-v3-base`, 5 epochs, class-balanced

## Imports and Configuration

In [ ]:
import json, random, copy
from sklearn.model_selection import train_test_split
import torch
from collections import Counter
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)
from sklearn.metrics import accuracy_score, f1_score

## Data Preparation

### Generate QP Verifier Dataset

In [ ]:
import json
import random
from sklearn.model_selection import train_test_split

# CONFIGURATION
INPUT = "/content/drive/MyDrive/llm-sr-project/700dataset.json"
OUT_TRAIN = "/content/drive/MyDrive/llm-sr-project/verifier_qp_train.jsonl"
OUT_DEV = "/content/drive/MyDrive/llm-sr-project/verifier_qp_dev.jsonl"
NEG_PER_POS = 1  # Number of negative samples to generate per positive
DEV_SIZE = 0.1   # Fraction of total data to allocate to dev set


def corrupt_question_parsing(qp):
    """
    Corrupts a valid question_parsing (QP) list by:
    - Randomly dropping one constraint (if list length > 1), or
    - Shuffling the order of constraints
    """
    qp2 = qp.copy()
    if random.random() < 0.5 and len(qp2) > 1:
        # drop one random item
        qp2.pop(random.randrange(len(qp2)))
    else:
        # shuffle the parsing
        random.shuffle(qp2)
    return qp2

def make_record(question, qp, label):
    """
    Formats a (question, question_parsing) pair into a binary classification example:
    - Premise: the raw question text
    - Hypothesis: serialized JSON version of QP
    - Label: 1 for valid, 0 for corrupted
    """
    premise = question
    hyp_qp = json.dumps(qp, ensure_ascii=False)
    hypothesis = f"QuestionParsing: {hyp_qp}"
    return {"premise": premise, "hypothesis": hypothesis, "label": label}

def main():
    # 1) Load positive (gold) examples from dataset
    with open(INPUT, "r", encoding="utf-8") as f:
        positives = json.load(f)

    # 2) For each gold example, generate one valid and several corrupted versions
    all_records = []
    for ex in positives:
        q = ex["question"]
        qp = ex["question_parsing"]

        # Add the original, valid (label=1)
        all_records.append(make_record(q, qp, 1))

        # Add corrupted examples (label=0)
        for _ in range(NEG_PER_POS):
            qp_bad = corrupt_question_parsing(qp)

            # Ensure the negative is actually different
            if qp_bad != qp:
                all_records.append(make_record(q, qp_bad, 0))

    # 3) Stratified train/dev split (preserve label ratio)
    train, dev = train_test_split(
        all_records,
        test_size=DEV_SIZE,
        random_state=42,
        stratify=[r["label"] for r in all_records]
    )

    # 4) Write train and dev splits as JSONL
    for path, split in [(OUT_TRAIN, train), (OUT_DEV, dev)]:
        with open(path, "w", encoding="utf-8") as f:
            for rec in split:
                f.write(json.dumps(rec, ensure_ascii=False) + "\n")

    print(f"✔︎ wrote {len(train)} train + {len(dev)} dev QP examples")

if __name__ == "__main__":
    main()

✔︎ wrote 694 train + 78 dev QP examples


### Define CoT Verifier Dataset

In [ ]:
import json
import random
import copy
from sklearn.model_selection import train_test_split

# CONFIGURATION
INPUT        = "/content/drive/MyDrive/llm-sr-project/700dataset.json"
OUT_TRAIN    = "/content/drive/MyDrive/llm-sr-project/verifier_cp_train.jsonl"
OUT_DEV      = "/content/drive/MyDrive/llm-sr-project/verifier_cp_dev.jsonl"
NEG_PER_POS  = 1    # Number of negative samples to generate per positive
DEV_SIZE     = 0.1  # Number of negative samples to generate per positive


def corrupt_cot_parsing(cp):
    """
    Corrupts a valid CoT parsing (list of statements with evidence and verification)
    by one of the following:
    - Flipping the verification field ("true" ↔ "false")
    - Swapping evidence between two statements
    - Dropping the evidence field from one step
    """
    cp2 = copy.deepcopy(cp)
    if not cp2:
        return cp2
    choice = random.choice(["flip", "swap", "drop_field"])
    if choice == "flip":
        idx = random.randrange(len(cp2))
        cur = cp2[idx].get("Verification", "false")
        cp2[idx]["Verification"] = "true" if cur=="false" else "false"
    elif choice == "swap" and len(cp2) >= 2:
        i, j = random.sample(range(len(cp2)), 2)
        cp2[i]["evidence"], cp2[j]["evidence"] = cp2[j].get("evidence"), cp2[i].get("evidence")
    else:
        idx = random.randrange(len(cp2))
        cp2[idx].pop("evidence", None)
    return cp2

def make_record(question, cot, conditions, cp, label):
    """
    Formats a (question, CoT, conditions, CoT parsing) pair into a binary classification example:
    - Premise: question + conditions + CoT text
    - Hypothesis: serialized JSON version of the CoT parsing
    - Label: 1 for valid, 0 for corrupted
    """
    cond_block = "\n".join(conditions)
    premise    = (
        f"Question:\n{question}\n\n"
        f"Conditions:\n{cond_block}\n\n"
        f"CoT:\n{cot}"
    )
    hyp_cp     = json.dumps(cp, ensure_ascii=False)
    hypothesis = f"CoTParsing: {hyp_cp}"
    return {"premise": premise, "hypothesis": hypothesis, "label": label}

def main():
    # 1) Load gold-labeled examples from dataset
    with open(INPUT, "r", encoding="utf-8") as f:
        positives = json.load(f)

    # 2) For each gold example, generate one valid and several corrupted versions
    all_records = []
    for ex in positives:
        q          = ex["question"]
        cot        = ex["cot"]
        qp         = ex["question_parsing"]
        conditions = qp[:-1] if len(qp) > 1 else qp
        cp         = ex["cot_parsing"]

        # Add the original, valid (label=1)
        all_records.append(make_record(q, cot, conditions, cp, 1))

        # Add corrupted examples (label=0)
        for _ in range(NEG_PER_POS):
            cp_bad = corrupt_cot_parsing(cp)
            if cp_bad != cp:
                all_records.append(make_record(q, cot, conditions, cp_bad, 0))

    # 3) Stratified train/dev split (preserve label ratio)
    train, dev = train_test_split(
        all_records,
        test_size=DEV_SIZE,
        random_state=42,
        stratify=[r["label"] for r in all_records]
    )

    # 4) Write train and dev splits as JSONL
    for path, split in [(OUT_TRAIN, train), (OUT_DEV, dev)]:
        with open(path, "w", encoding="utf-8") as f:
            for rec in split:
                f.write(json.dumps(rec, ensure_ascii=False) + "\n")

    print(f"✔︎ wrote {len(train)} train + {len(dev)} dev CP examples")

if __name__ == "__main__":
    main()

✔︎ wrote 703 train + 79 dev CP examples


## Train QP Verifier

In [ ]:
import torch
import random
from collections import Counter
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)
import json
from datasets import Dataset, DatasetDict
from sklearn.metrics import accuracy_score, f1_score

# CONFIGURATION
MODEL_NAME       = "microsoft/deberta-v3-base"
TRAIN_FILE_QP    = "/content/drive/MyDrive/llm-sr-project/verifier_qp_train.jsonl"
DEV_FILE_QP      = "/content/drive/MyDrive/llm-sr-project/verifier_qp_dev.jsonl"
OUTPUT_DIR       = "/content/drive/MyDrive/deberta-qparse-verifier"
BATCH_SIZE       = 4
NUM_EPOCHS       = 5
LEARNING_RATE    = 1e-5
DEVICE           = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# 1) Load Dataset
def load_jsonl(path):
    with open(path, "r") as f:
       return [json.loads(line) for line in f]

train_data = load_jsonl(TRAIN_FILE_QP)
val_data   = load_jsonl(DEV_FILE_QP)

ds = DatasetDict({
    "train": Dataset.from_list(train_data),
    "validation": Dataset.from_list(val_data)
})

#ds = load_dataset("json", data_files={"train": TRAIN_FILE_QP, "validation": DEV_FILE_QP})

# 2) Balance a split function
def balance_split(split_ds):
    labels   = split_ds["label"]
    idxs_neg = [i for i,l in enumerate(labels) if l == 0]
    idxs_pos = [i for i,l in enumerate(labels) if l == 1]

    random.seed(42)
    # down-sample the larger class
    if len(idxs_neg) > len(idxs_pos):
        idxs_neg = random.sample(idxs_neg, len(idxs_pos))
    else:
        idxs_pos = random.sample(idxs_pos, len(idxs_neg))

    # combine & shuffle
    balanced_idxs = idxs_neg + idxs_pos
    random.shuffle(balanced_idxs)
    return split_ds.select(balanced_idxs)

# 3) Apply to both splits
ds["train"]      = balance_split(ds["train"])
ds["validation"] = balance_split(ds["validation"])

print("Balanced train counts:     ", Counter(ds["train"]["label"]))
print("Balanced validation counts:", Counter(ds["validation"]["label"]))


# 2) Tokenizer & Model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model     = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
model.to(DEVICE)

data_collator = DataCollatorWithPadding(tokenizer)

def preprocess(examples):
    enc = tokenizer(
        examples["premise"],
        examples["hypothesis"],
        truncation=True
    )
    enc["labels"] = examples["label"]
    return enc

tok_ds = ds.map(preprocess, batched=True, remove_columns=ds["train"].column_names)

# 3) Metrics
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1":       f1_score(labels, preds),
    }

# 4) TrainingArguments & Trainer
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE * 2,
    learning_rate=LEARNING_RATE,
    num_train_epochs=NUM_EPOCHS,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    logging_steps=50,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tok_ds["train"],
    eval_dataset=tok_ds["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# 5) Train & Save
trainer.train()
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print("✅ QP verifier trained and saved to", OUTPUT_DIR)

Balanced train counts:      Counter({0: 343, 1: 343})
Balanced validation counts: Counter({0: 38, 1: 38})


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

...

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/76 [00:00<?, ? examples/s]

<ipython-input-4-87f0c796dbbc>:111: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.683000,0.657719,0.736842,0.772727
2,0.540600,0.563493,0.736842,0.772727
3,0.563400,0.562213,0.750000,0.781609
4,0.397500,0.794427,0.736842,0.767442
5,0.379800,0.855469,0.750000,0.771084


✅ QP verifier trained and saved to /content/drive/MyDrive/deberta-qparse-verifier


In [33]:
!rm -rf "/content/drive/MyDrive/deberta-qparse-verifier/checkpoint-"*

In [34]:
!ls "/content/drive/MyDrive/deberta-qparse-verifier"

added_tokens.json  special_tokens_map.json  tokenizer.json
config.json	   spm.model		    training_args.bin
model.safetensors  tokenizer_config.json


In [35]:
!zip -r /content/drive/MyDrive/deberta-qparse-verifier.zip "/content/drive/MyDrive/deberta-qparse-verifier"

updating: content/drive/MyDrive/deberta-qparse-verifier/ (stored 0%)
updating: content/drive/MyDrive/deberta-qparse-verifier/config.json (deflated 54%)
updating: content/drive/MyDrive/deberta-qparse-verifier/model.safetensors (deflated 25%)
updating: content/drive/MyDrive/deberta-qparse-verifier/tokenizer_config.json (deflated 73%)
updating: content/drive/MyDrive/deberta-qparse-verifier/training_args.bin (deflated 51%)
updating: content/drive/MyDrive/deberta-qparse-verifier/added_tokens.json (stored 0%)
updating: content/drive/MyDrive/deberta-qparse-verifier/spm.model (deflated 50%)
updating: content/drive/MyDrive/deberta-qparse-verifier/special_tokens_map.json (deflated 50%)
updating: content/drive/MyDrive/deberta-qparse-verifier/tokenizer.json (deflated 77%)


## Train CoT Verifier

In [26]:
import torch
import random
from collections import Counter
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)
from sklearn.metrics import accuracy_score, f1_score

# CONFIGURATION
MODEL_NAME       = "microsoft/deberta-v3-base"
TRAIN_FILE_CP    = "/content/drive/MyDrive/llm-sr-project/verifier_cp_train.jsonl"
DEV_FILE_CP      = "/content/drive/MyDrive/llm-sr-project/verifier_cp_dev.jsonl"
OUTPUT_DIR       = "/content/drive/MyDrive/deberta-cotparse-verifier"
BATCH_SIZE       = 4
NUM_EPOCHS       = 5
LEARNING_RATE    = 1e-5
DEVICE           = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1) Load Dataset
def load_jsonl(path):
    with open(path, "r") as f:
       return [json.loads(line) for line in f]

train_data = load_jsonl(TRAIN_FILE_QP)
val_data   = load_jsonl(DEV_FILE_QP)

ds = DatasetDict({
    "train": Dataset.from_list(train_data),
    "validation": Dataset.from_list(val_data)
})


#ds = load_dataset("json",data_files={"train": TRAIN_FILE_CP, "validation": DEV_FILE_CP})

# 2) Balance a split function
def balance_split(split_ds):
    labels   = split_ds["label"]
    idxs_neg = [i for i,l in enumerate(labels) if l == 0]


    idxs_pos = [i for i,l in enumerate(labels) if l == 1]

    random.seed(42)
    # down-sample the larger class
    if len(idxs_neg) > len(idxs_pos):
        idxs_neg = random.sample(idxs_neg, len(idxs_pos))
    else:
        idxs_pos = random.sample(idxs_pos, len(idxs_neg))

    # combine & shuffle
    balanced_idxs = idxs_neg + idxs_pos
    random.shuffle(balanced_idxs)
    return split_ds.select(balanced_idxs)

# 3) Apply to both splits
ds["train"]      = balance_split(ds["train"])
ds["validation"] = balance_split(ds["validation"])

print("Balanced train counts:     ", Counter(ds["train"]["label"]))
print("Balanced validation counts:", Counter(ds["validation"]["label"]))

# 4) Tokenizer & Model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model     = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
model.to(DEVICE)

data_collator = DataCollatorWithPadding(tokenizer)

def preprocess(examples):
    enc = tokenizer(
        examples["premise"],
        examples["hypothesis"],
        truncation=True
    )
    enc["labels"] = examples["label"]
    return enc

tok_ds = ds.map(preprocess, batched=True, remove_columns=ds["train"].column_names)

# 5) Metrics
def compute_metrics(p):
    preds  = p.predictions.argmax(-1)
    labels = p.label_ids
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1":       f1_score(labels, preds),
    }

# 6) TrainingArguments & Trainer
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE * 2,
    learning_rate=LEARNING_RATE,
    num_train_epochs=NUM_EPOCHS,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    logging_steps=50,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tok_ds["train"],
    eval_dataset=tok_ds["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# 7) Train & Save
trainer.train()
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print("✅ CP verifier trained and saved to", OUTPUT_DIR)

Balanced train counts:      Counter({0: 343, 1: 343})
Balanced validation counts: Counter({0: 38, 1: 38})


/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/686 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/76 [00:00<?, ? examples/s]

<ipython-input-26-cede48a6fa05>:110: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.693300,0.693211,0.500000,0.000000
2,0.651300,0.576170,0.697368,0.757895
3,0.552300,0.537043,0.763158,0.763158
4,0.372300,0.652095,0.763158,0.775000
5,0.399100,0.721006,0.789474,0.804878


✅ CP verifier trained and saved to /content/drive/MyDrive/deberta-cotparse-verifier


In [30]:
!rm -rf "/content/drive/MyDrive/deberta-cotparse-verifier/checkpoint-"*

In [31]:
!ls "/content/drive/MyDrive/deberta-cotparse-verifier"

added_tokens.json  special_tokens_map.json  tokenizer.json
config.json	   spm.model		    training_args.bin
model.safetensors  tokenizer_config.json


In [32]:
!zip -r /content/drive/MyDrive/deberta-cotparse-verifier.zip "/content/drive/MyDrive/deberta-cotparse-verifier"

updating: content/drive/MyDrive/deberta-cotparse-verifier/ (stored 0%)
updating: content/drive/MyDrive/deberta-cotparse-verifier/config.json (deflated 54%)
updating: content/drive/MyDrive/deberta-cotparse-verifier/model.safetensors (deflated 25%)
updating: content/drive/MyDrive/deberta-cotparse-verifier/training_args.bin (deflated 52%)
updating: content/drive/MyDrive/deberta-cotparse-verifier/special_tokens_map.json (deflated 50%)
updating: content/drive/MyDrive/deberta-cotparse-verifier/tokenizer_config.json (deflated 73%)
updating: content/drive/MyDrive/deberta-cotparse-verifier/spm.model (deflated 50%)
updating: content/drive/MyDrive/deberta-cotparse-verifier/tokenizer.json (deflated 77%)
updating: content/drive/MyDrive/deberta-cotparse-verifier/added_tokens.json (stored 0%)


## Next Steps

- Both verifiers have been saved to:
  - `deberta-qparse-verifier/`
  - `deberta-cotparse-verifier/`

- In the next notebook, we will integrate these two models into our hybrid inference pipeline to rerank and validate Chain-of-Thought parses.
